# Automated Classification of Argumentative Elements in Student Writing

## Tabla de Contenidos
1. [Introduction](#Introducción)
2. [1. Selección de Algoritmos](#1-Selección-de-Algoritmos)
3. [2. Preprocesamiento e Ingeniería de Características](#2-Preprocesamiento-e-Ingeniería-de-Características)
4. [3. Implementación de Modelos](#3-Implementación-de-Modelos)
5. [4. Evaluación y Comparación de Modelos](#4-Evaluación-y-Comparación-de-Modelos)
6. [5. Visualización y Comunicación de Resultados](#5-Visualización-y-Comunicación-de-Resultados)
7. [Conclusiones](#Conclusiones)
8. [Referencias](#Referencias)


## Introducción

El objetivo de este proyecto se basa en crear modelos de machine para clasificar los elementos argumentativos en los textos escritos por estudiantes como "efectivos", "adecuados" o "ineficaces". Esta clasificación automatizada tiene como objetivo proporcionar una mejor retroalimentación a los estudiantes, para así ellos puedan mejorar sus habilidades de escritura.

---

## 1. Selección de Algoritmos

### Investigación y Selección de Algoritmos

Para abordar el problema de clasificación de elementos argumentativos en textos de estudiantes, se seleccionaron los siguientes cinco algoritmos de Machine Learning:

1. **Logistic Regression**
2. **Random Forest Classifier**
3. **XGBoost**
4. **Red Neuronal (Neural Network) con Keras**
5. **Red Neuronal (Neural Network) con PyTorch**

### Justificación de la Selección

1. **Logistic Regression**:
   - **Características**: Modelo lineal simple que es eficiente y fácil de interpretar.
   - **Justificación**: Adecuado para problemas de clasificación multiclase y sirve como un buen punto de referencia.
   - **Referencia**: [Hosmer, D.W., Lemeshow, S. (2000). Applied Logistic Regression. Wiley.]

2. **Random Forest Classifier**:
   - **Características**: Ensamble de árboles de decisión que maneja bien las interacciones no lineales y la importancia de características.
   - **Justificación**: Eficaz para manejar conjuntos de datos con características complejas y evita el sobreajuste.
   - **Referencia**: [Breiman, L. (2001). Random Forests. Machine Learning, 45(1), 5-32.]

3. **XGBoost**:
   - **Características**: Algoritmo de gradiente potenciado que es altamente eficiente y preciso, especialmente en problemas de clasificación y regresión.
   - **Justificación**: Excelente para manejar características de alta dimensionalidad y ofrece mecanismos avanzados de regularización para evitar el sobreajuste.
   - **Referencia**: [Chen, T., & Guestrin, C. (2016). XGBoost: A Scalable Tree Boosting System. Proceedings of the 22nd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining.]

4. **Red Neuronal (Neural Network) con Keras**:
   - **Características**: Modelos capaces de capturar relaciones complejas y no lineales en los datos mediante múltiples capas de neuronas.
   - **Justificación**: Adecuada para procesamiento de texto y puede aprovechar representaciones de características profundas para mejorar la clasificación.
   - **Referencia**: [Goodfellow, I., Bengio, Y., & Courville, A. (2016). Deep Learning. MIT Press.]

5. **Red Neuronal (Neural Network) con PyTorch**:
   - **Características**: Similar a las redes neuronales con Keras, pero con mayor flexibilidad y control en el proceso de entrenamiento.
   - **Justificación**: Permite una personalización más detallada del modelo y el proceso de entrenamiento, lo que puede mejorar el rendimiento en tareas específicas.
   - **Referencia**: [Paszke, A., et al. (2019). PyTorch: An Imperative Style, High-Performance Deep Learning Library. Advances in Neural Information Processing Systems.]

---

## 2. Preprocesamiento e Ingeniería de Características

### Carga de Datos

In [2]:
import pandas as pd

# Cargar los datos preprocesados
train_df = pd.read_csv('data/train_preprocessed.csv')
test_df = pd.read_csv('data/test.csv')

# Visualizar las primeras filas del conjunto de entrenamiento
train_df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,text_length,text
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,317,hi im isaac im going writing face mars natural...
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,210,perspective think face natural landform dont t...
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,105,think face natural landform life mars descover...
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,362,life mars would know reason think natural land...
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,101,people thought face formed alieans thought lif...


### Limpieza de Datos

Aunque ya se ha realizado un limpieza previa, esta es solo una medida de seguridad.

In [3]:
# Verificar valores nulos
train_df.isnull().sum()

# Rellenar o eliminar valores nulos si es necesario
train_df = train_df.dropna()

### Ingeniería de Características (Feature Engineering)

1. **Tokenización y Normalización del Texto**:
   - Convertir texto a minúsculas.
   - Eliminar puntuación y caracteres especiales.
   - Tokenizar palabras.

   > Esto fue realizado en la etapa previa (Normalización)

2. **Vectorización**:
   - Utilizar TF-IDF para transformar el texto en vectores numéricos.

3. **Características Adicionales**:
   - Longitud del texto (`text_length`).
   - Número de palabras clave específicas relacionadas con elementos argumentativos.

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

# Vectorización del texto
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_text = tfidf.fit_transform(train_df['text'])

# Escalar la característica de longitud del texto
scaler = StandardScaler()
X_length = scaler.fit_transform(train_df[['text_length']])

# Concatenar características
import scipy.sparse as sp
X = sp.hstack([X_text, X_length])

# Variable objetivo
y = train_df['discourse_effectiveness']

### Justificación de las Técnicas Aplicadas

- **TF-IDF**: Captura la importancia relativa de las palabras en el contexto del corpus, lo que es esencial para entender la efectividad argumentativa.
- **Escalado de Características**: Normaliza la longitud del texto para que no domine otras características durante el entrenamiento del modelo.

---

## 3. Implementación de Modelos

### División de Datos

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

### Definición de Modelos

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder

# Inicializar modelos tradicionales
log_reg = LogisticRegression(multi_class='multinomial', max_iter=1000, random_state=42)
rand_forest = RandomForestClassifier(n_estimators=100, random_state=42)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

# Preparar datos para la Red Neuronal con Keras
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_val_enc = le.transform(y_val)

# Definir el modelo de Red Neuronal con Keras
def create_nn_model(input_dim, num_classes):
    model = Sequential()
    model.add(Dense(512, input_dim=input_dim, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

num_classes = y.nunique()
nn_model_keras = create_nn_model(X_train.shape[1], num_classes)
nn_model_keras.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Preparar datos para la Red Neuronal con PyTorch
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X.toarray().astype('float32')
        self.y = y.astype('int64')

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset_pytorch = TextDataset(X_train, y_train_enc)
val_dataset_pytorch = TextDataset(X_val, y_val_enc)

train_loader_pytorch = DataLoader(train_dataset_pytorch, batch_size=128, shuffle=True)
val_loader_pytorch = DataLoader(val_dataset_pytorch, batch_size=128, shuffle=False)

# Definir el modelo de Red Neuronal con PyTorch
class NeuralNetPyTorch(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NeuralNetPyTorch, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 256)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

nn_model_pytorch = NeuralNetPyTorch(X_train.shape[1], num_classes)

c:\Users\chama\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
